# Put it all together and write a function to record notices of allocation

Fri 14 Sep, 2018

In [1]:
import numpy as np
import pandas as pd

## Create the following tables:

**members:** member_id, name

**membership-shares:** date, name, amount

**preferred-shares:** date, name, amount

**other-equity:** date, name, amount

In [2]:
members_df = pd.DataFrame([
    [1, 'Alice'],
    [2, 'Bob'],
    [3, 'Candice'],
    [4, 'Darwin'],
    [5, 'Eve'],
    [6, 'Federico'],
    [7, 'Gabi'],
    [8, 'HAL'],
    [9, 'Irene'],
    [10, 'Jules']
    #,[11, 'Karma']
], columns = ['member_id', 'name'])
members_df

,member_id,name
0,1,Alice
1,2,Bob
2,3,Candice
3,4,Darwin
4,5,Eve
5,6,Federico
6,7,Gabi
7,8,HAL
8,9,Irene
9,10,Jules


In [3]:
membership_df = pd.DataFrame([
    ["2/15/2015", 'Bob', 125],
    ["2/15/2015", 'Candice', 125],
    ["2/15/2015", 'Alice', 125],
    ["8/7/2015", 'Darwin', 125],
    ["11/2/2015", 'Eve', 125],
    ["1/20/2016", 'Federico', 125],
    ["4/17/2016", 'Gabi', 125],
    ["7/5/2016", 'HAL', 125],
    ["3/23/2017", 'Irene', 125],
    ["5/24/2017", 'Jules', 125],
], columns = ['date', 'name', 'amount'])
membership_df

,date,name,amount
0,2/15/2015,Bob,125
1,2/15/2015,Candice,125
2,2/15/2015,Alice,125
3,8/7/2015,Darwin,125
4,11/2/2015,Eve,125
5,1/20/2016,Federico,125
6,4/17/2016,Gabi,125
7,7/5/2016,HAL,125
8,3/23/2017,Irene,125
9,5/24/2017,Jules,125


In [4]:
preferred_df = pd.DataFrame([
    ["6/21/2015", 'Candice', 2500],
    ["6/21/2015", 'Bob', 1500],
    ["9/14/2015", 'Darwin', 4000],
    ["3/6/2016", 'Eve', 15000],
    ["4/27/2016", 'Federico', 3000],
    ["11/9/2016", 'Bob', 3500],
    ["1/10/2017", 'HAL', 2000],
    ["5/18/2017", 'Irene', 20000],
    ["7/9/2017", 'Eve', -5000],
], columns = ['date', 'name', 'amount'])
preferred_df

,date,name,amount
0,6/21/2015,Candice,2500
1,6/21/2015,Bob,1500
2,9/14/2015,Darwin,4000
3,3/6/2016,Eve,15000
4,4/27/2016,Federico,3000
5,11/9/2016,Bob,3500
6,1/10/2017,HAL,2000
7,5/18/2017,Irene,20000
8,7/9/2017,Eve,-5000


In [5]:
other_equity_df = pd.DataFrame([
    ["3/14/2015", "Alice", 200],
    ["3/14/2015", 'Bob', 200],
    ["12/5/2015", 'Candice', 1000],
    ["9/30/2016", 'Candice', -500],
    ["12/10/2016", 'Gabi', 500],
    ["2/28/2017", 'Bob', -200],
    ["4/25/2017", 'Candice', -500],
    ["10/12/2017", 'Alice', 250]
], columns = ['date', 'name', 'amount'])
other_equity_df

,date,name,amount
0,3/14/2015,Alice,200
1,3/14/2015,Bob,200
2,12/5/2015,Candice,1000
3,9/30/2016,Candice,-500
4,12/10/2016,Gabi,500
5,2/28/2017,Bob,-200
6,4/25/2017,Candice,-500
7,10/12/2017,Alice,250


## Collect functions from `load_tables` and `allocations` notebooks

And make some edits...

### Functions from `load_tables` notebook for generating input dataframes

In [65]:
def get_contributions_df(members_df, membership_df, preferred_df, other_equity_df):
    """Combines data from all four data tables into one list of transactions.
    """
    #This version modifies the input dataframes:
    #membership_df['type'] = 'membership'
    #preferred_df['type'] = 'preferred'
    #other_equity_df['type'] = 'other'
    
    #Instead, use assign to make a copy of the dataframes rather than modifying them in place:
    membership_df = membership_df.assign(type='membership')
    preferred_df = preferred_df.assign(type='preferred')
    other_equity_df = other_equity_df.assign(type='other')
    
    equity_dfs = [membership_df, preferred_df, other_equity_df]
    contributions_df = pd.concat(equity_dfs, ignore_index=True)
    
    contributions_df = members_df.merge(contributions_df, on='name')
    #contributions_df.rename(columns={'id': 'member_id'}, inplace=True)
    
    return contributions_df

def split_by_year(contributions_df, current_year):
    """Splits a list of transactions into 3 lists:
    one for the current year, one for past years, and one for future years.
    """
    year = pd.to_datetime(contributions_df['date']).apply(lambda date: date.year)
    past = contributions_df[year < current_year]
    current = contributions_df[year == current_year]
    future = contributions_df[year > current_year]
    return past, current, future

def get_equity_df(members_df, contributions_df):
    """Gets a dataframe displaying each member's different types of equity, based on the sum
    of the amounts in the given list of contributions.
    """
    if len(contributions_df) == 0:
        equity_df = members_df.assign(membership=0,preferred=0,other=0,equity=0)
    else:
        equity_df = contributions_df.pivot_table(
            index=['name'], columns=['type'], values='amount', aggfunc=np.sum, fill_value=0)
        equity_df = members_df.merge(equity_df, left_on='name', right_index=True, how='outer')
        equity_df.fillna(0, inplace=True)
        equity_df['equity'] = equity_df[['membership', 'preferred', 'other']].sum(axis=1)
        
    return equity_df

In [66]:
past, present, future = split_by_year(contributions_df, 2016)
get_equity_df(members_df, past)

,member_id,name,membership,other,preferred,equity
0,1,Alice,125.0,200.0,0.0,325.0
1,2,Bob,125.0,200.0,1500.0,1825.0
2,3,Candice,125.0,1000.0,2500.0,3625.0
3,4,Darwin,125.0,0.0,4000.0,4125.0
4,5,Eve,125.0,0.0,0.0,125.0
5,6,Federico,0.0,0.0,0.0,0.0
6,7,Gabi,0.0,0.0,0.0,0.0
7,8,HAL,0.0,0.0,0.0,0.0
8,9,Irene,0.0,0.0,0.0,0.0
9,10,Jules,0.0,0.0,0.0,0.0


In [67]:
split_by_year(contributions_df, 2015)

(Empty DataFrame
 Columns: [member_id, name, date, amount, type]
 Index: [],     member_id     name       date  amount        type
 0           1    Alice  2/15/2015     125  membership
 1           1    Alice  3/14/2015     200       other
 3           2      Bob  2/15/2015     125  membership
 4           2      Bob  6/21/2015    1500   preferred
 6           2      Bob  3/14/2015     200       other
 8           3  Candice  2/15/2015     125  membership
 9           3  Candice  6/21/2015    2500   preferred
 10          3  Candice  12/5/2015    1000       other
 13          4   Darwin   8/7/2015     125  membership
 14          4   Darwin  9/14/2015    4000   preferred
 15          5      Eve  11/2/2015     125  membership,     member_id      name        date  amount        type
 2           1     Alice  10/12/2017     250       other
 5           2       Bob   11/9/2016    3500   preferred
 7           2       Bob   2/28/2017    -200       other
 11          3   Candice   9/30/2016

In [69]:
past, present, future = split_by_year(contributions_df, 2015)
past

,member_id,name,date,amount,type


In [70]:
past.index

Int64Index([], dtype='int64')

In [71]:
members_df.index

RangeIndex(start=0, stop=10, step=1)

In [72]:
get_equity_df(members_df, past)

,member_id,name,membership,preferred,other,equity
0,1,Alice,0,0,0,0
1,2,Bob,0,0,0,0
2,3,Candice,0,0,0,0
3,4,Darwin,0,0,0,0
4,5,Eve,0,0,0,0
5,6,Federico,0,0,0,0
6,7,Gabi,0,0,0,0
7,8,HAL,0,0,0,0
8,9,Irene,0,0,0,0
9,10,Jules,0,0,0,0


### Functions from `allocations` notebook for computing patronage allocations

In [7]:
yak = 'yak'
preferred_df.assign(yak=yak)

,date,name,amount,yak
0,6/21/2015,Candice,2500,yak
1,6/21/2015,Bob,1500,yak
2,9/14/2015,Darwin,4000,yak
3,3/6/2016,Eve,15000,yak
4,4/27/2016,Federico,3000,yak
5,11/9/2016,Bob,3500,yak
6,1/10/2017,HAL,2000,yak
7,5/18/2017,Irene,20000,yak
8,7/9/2017,Eve,-5000,yak


In [27]:
def fraction_year_remaining(date):
    """Computes the fraction of the year remaining from a given date.
    """
    date = pd.to_datetime(date)
    offset = pd.tseries.offsets.YearBegin()
    year = date - offset
    next_year = date + offset
    #print(offset, year, next_year)
    return (next_year - date).days / (next_year - year).days

def compute_transaction_patronage(contributions_df):
    """Compute patronage for each transaction based on amount and fraction of year remaining.
    """
    patronage = contributions_df['amount']*contributions_df['date'].apply(fraction_year_remaining)
    return contributions_df.assign(patronage=patronage) #This makes a copy and adds a new column

def compute_new_patronage(new_equity_df):
    """Compute each member's patronage from new contributions for the current year.
    """
    new_equity_df = compute_transaction_patronage(new_equity_df)
    return new_equity_df[['member_id','patronage']].groupby(by='member_id').sum()

def compute_patronage(old_equity_df, new_equity_df):
    """Compute total patronage for each member from new contributions for the current year
        and existing equity from previous years.
    """
    patronage_df = old_equity_df.set_index('member_id')[['name', 'equity']]
    patronage_df.rename(columns={'equity': 'old_patronage'}, inplace=True)
    
#     new_equity_df = compute_transaction_patronage(new_equity_df)
#     new_equity_df[['member_id','patronage']].groupby(by='member_id').sum()
    
    patronage_df['new_patronage'] = compute_new_patronage(new_equity_df)['patronage']
    # If there were members with no contributions this year, set their new patronage to 0 (would be NaN).
    patronage_df.fillna(0, inplace=True)
    
    patronage_df['patronage'] = patronage_df['old_patronage'] + patronage_df['new_patronage']
    patronage_df['proportionate_patronage'] = patronage_df['patronage'] / patronage_df['patronage'].sum()
    
    return patronage_df

def compute_patronage_for_year(members_df, contributions_df, year):
    """Computes each member's patronage for the specified year."""
    old_equity_df, new_equity_df, _ = split_by_year(contributions_df, year)
    old_equity_df = get_equity_df(members_df, old_equity_df)
    return compute_patronage(old_equity_df, new_equity_df)

def compute_dividends(patronage_df, profit, proportion_individual=0.5, rounded=True):
    """Compute each member's dividend based on patronage for the year.
    """ 
    dividend_df = patronage_df[['name', 'proportionate_patronage']].copy()
    
    #Compute individual patronage allocations
    dividend_df['dividend'] = dividend_df['proportionate_patronage'] * profit * proportion_individual
    if rounded:
        dividend_df['dividend'] = np.round(dividend_df['dividend'], 2)
    
    # To account for rounding amounts to the nearest cent, we add up the individual dividends
    # to get the actual amount allocated to individual net income. Then we subtract this amount
    # from the total profit to get the collective net income.
    indiv_profit = dividend_df['dividend'].sum()
    collective_profit = profit - indiv_profit
    
    # We reserve member_id=0 for the collective account (or we could simply use names as keys)
    dividend_df.loc[0] = pd.Series({
        'name': 'CollectiveAcct',
        'proportionate_patronage': collective_profit / indiv_profit,
        'dividend': collective_profit
    })
    return dividend_df

def compute_allocations(dividend_df,
                        year,
                        first_year_proportion=0.5,
                        n_years=3,
                        distribution = None,
                        irregular_payment='last'):
    """Computes allocations over next n_years years after the dividend year,
    or according to the given payment distribution.
    """
    
    #Currently we assume below that either all dividends will be positive or all will
    #be negative, depending on the overall profit.
    #Conceivably there could be a situation where some dividends could be positive
    #and some could be negative, in which case we'd need two distribution arrays,
    #but I can't currently think of why we would want to allow that.
    
    #If no explicit payment distribution was passed, we need to create one.
    if distribution is None:
        #Check whether the total profit is positive or negative, and create
        #the appropriate default distribution.
        if dividend_df['dividend'].sum() >= 0:
            #profit >= 0
            #Default for a positive dividend is to evenly evenly divide what's left over after
            #the first year over the remaining n-1 years.
            if n_years == 1:
                #Avoid divide by zero error.
                distribution = [1]
            else:
                #n_years > 1
                distribution = [(1-first_year_proportion) / (n_years-1) for _ in range(n_years)]
                distribution[0] = first_year_proportion
        else:
            #profit < 0
            #Default for a negative dividend is to evenly divide it over all n years.
            distribution = [1.0 / n_years for _ in range(n_years)]
            
    #Convert the distribution from a list to a numpy array to perform math with it.
    distribution = np.array(distribution)
    
    #In case the distribution was explicitly passed, make sure n_years matches the actual length.
    n_years = len(distribution)
    
    #Create a new DataFrame for the allocations by dropping the collective account and
    #the proportionate patronage column from dividend_df.
    allocation_df = dividend_df.drop(index=0, columns='proportionate_patronage')
    
    #Use broadcasting to compute all dividends with one multiplication,
    #and round them to nearest cent.
    allocations = (distribution.reshape(1,-1) * allocation_df['dividend'].values.reshape(-1,1)).round(2)
    
    #Adjust the first or last payment to account for rounding, by replacing the
    #first or last column, respectively, with the result of subtracting
    #the sum of the remaining payments from the member's actual dividend.
    #Or if they passed a specific year, adjust the payment for that year.
    if irregular_payment == 'first': irregular_payment = 1
    elif irregular_payment == 'last': irregular_payment = n_years
    elif 1 <= irregular_payment <= n_years: pass
    else: raise ValueError(
        "irregular_payment must be 'first', 'last', or an integer between 1 and n_years (inclusive).")
        
    sum_of_remaining = (allocations[:,:irregular_payment-1].sum(axis=1)
                            + allocations[:,irregular_payment:].sum(axis=1))
    allocations[:,irregular_payment-1] = np.round(allocation_df['dividend'] - sum_of_remaining, 2)
        
    #Create new column labels for the years year+1, year+2,...,year+n
    new_columns = [str(y) for y in range(year+1, year+n_years+1)]
    
    #Concatenate the existing allocation dataframes horizontally with the computed allocations.
    allocation_df = pd.concat([allocation_df,
                              pd.DataFrame(allocations, index=allocation_df.index, columns=new_columns)],
                             axis=1)
    
    #Rename the 'dividend' column.
    allocation_df.rename(columns={'dividend': str(year)+'_dividend'}, inplace=True)
    
    return allocation_df

## Test the functions

In [13]:
contributions_df = get_contributions_df(members_df, membership_df, preferred_df, other_equity_df)
compute_transaction_patronage(contributions_df)

,member_id,name,date,amount,type,patronage
0,1,Alice,2/15/2015,125,membership,109.589041
1,1,Alice,3/14/2015,200,other,160.547945
2,1,Alice,10/12/2017,250,other,55.479452
3,2,Bob,2/15/2015,125,membership,109.589041
4,2,Bob,6/21/2015,1500,preferred,797.260274
5,2,Bob,11/9/2016,3500,preferred,506.830601
6,2,Bob,3/14/2015,200,other,160.547945
7,2,Bob,2/28/2017,-200,other,-168.219178
8,3,Candice,2/15/2015,125,membership,109.589041
9,3,Candice,6/21/2015,2500,preferred,1328.767123


In [14]:
compute_new_patronage(contributions_df)

,patronage
member_id,
1,325.616438
2,1406.008683
3,1041.443970
4,1244.863014
5,9945.654615
6,2159.494536
7,118.510929
8,2012.160341
9,12590.410959


In [29]:
compute_patronage_for_year(members_df, contributions_df, 2017)

,name,old_patronage,new_patronage,patronage,proportionate_patronage
member_id,,,,,
1,Alice,325.0,55.479452,380.479452,0.008717
2,Bob,5325.0,-168.219178,5156.780822,0.118140
3,Candice,3125.0,-343.835616,2781.164384,0.063716
4,Darwin,4125.0,0.000000,4125.000000,0.094503
5,Eve,15125.0,-2410.958904,12714.041096,0.291275
6,Federico,3125.0,0.000000,3125.000000,0.071593
7,Gabi,625.0,0.000000,625.000000,0.014319
8,HAL,125.0,1950.684932,2075.684932,0.047553
9,Irene,0.0,12590.410959,12590.410959,0.288443


In [63]:
compute_patronage_for_year(members_df, contributions_df, 2015)

,name,old_patronage,new_patronage,patronage,proportionate_patronage
member_id,,,,,
1,Alice,0,270.136986,270.136986,0.065643
2,Bob,0,1067.397260,1067.397260,0.259375
3,Candice,0,1512.328767,1512.328767,0.367492
4,Darwin,0,1244.863014,1244.863014,0.302498
5,Eve,0,20.547945,20.547945,0.004993
6,Federico,0,0.000000,0.000000,0.000000
7,Gabi,0,0.000000,0.000000,0.000000
8,HAL,0,0.000000,0.000000,0.000000
9,Irene,0,0.000000,0.000000,0.000000


In [64]:
compute_patronage_for_year(members_df, contributions_df, 2016)

,name,old_patronage,new_patronage,patronage,proportionate_patronage
member_id,,,,,
1,Alice,325.0,0.000000,325.000000,0.012958
2,Bob,1825.0,506.830601,2331.830601,0.092974
3,Candice,3625.0,-127.049180,3497.950820,0.139470
4,Darwin,4125.0,0.000000,4125.000000,0.164472
5,Eve,125.0,12336.065574,12461.065574,0.496846
6,Federico,0.0,2159.494536,2159.494536,0.086103
7,Gabi,0.0,118.510929,118.510929,0.004725
8,HAL,0.0,61.475410,61.475410,0.002451
9,Irene,0.0,0.000000,0.000000,0.000000


In [78]:
patronage_df = compute_patronage_for_year(members_df, contributions_df, 2015)
dividend_df = compute_dividends(patronage_df, 378.92)
allocations2015 = compute_allocations(dividend_df, 2015)
allocations2015

,name,2015_dividend,2016,2017,2018
member_id,,,,,
1,Alice,12.44,6.22,3.11,3.11
2,Bob,49.14,24.57,12.28,12.29
3,Candice,69.62,34.81,17.40,17.41
4,Darwin,57.31,28.66,14.33,14.32
5,Eve,0.95,0.48,0.24,0.23
6,Federico,0.00,0.00,0.00,0.00
7,Gabi,0.00,0.00,0.00,0.00
8,HAL,0.00,0.00,0.00,0.00
9,Irene,0.00,0.00,0.00,0.00


In [79]:
allocations2015.sum()

name             AliceBobCandiceDarwinEveFedericoGabiHALIreneJules
2015_dividend                                               189.46
2016                                                         94.74
2017                                                         47.36
2018                                                         47.36
dtype: object

In [81]:
patronage_df = compute_patronage_for_year(members_df, contributions_df, 2016)
dividend_df = compute_dividends(patronage_df, 953.53)
allocations2016 = compute_allocations(dividend_df, 2016)
allocations2016

,name,2016_dividend,2017,2018,2019
member_id,,,,,
1,Alice,6.18,3.09,1.54,1.55
2,Bob,44.33,22.16,11.08,11.09
3,Candice,66.49,33.24,16.62,16.63
4,Darwin,78.41,39.20,19.60,19.61
5,Eve,236.88,118.44,59.22,59.22
6,Federico,41.05,20.52,10.26,10.27
7,Gabi,2.25,1.12,0.56,0.57
8,HAL,1.17,0.58,0.29,0.30
9,Irene,0.00,0.00,0.00,0.00


In [82]:
allocations2016.sum()

name             AliceBobCandiceDarwinEveFedericoGabiHALIreneJules
2016_dividend                                               476.76
2017                                                        238.35
2018                                                        119.17
2019                                                        119.24
dtype: object

In [83]:
patronage_df = compute_patronage_for_year(members_df, contributions_df, 2017)
dividend_df = compute_dividends(patronage_df, 1745.77)
allocations2017 = compute_allocations(dividend_df, 2017)
allocations2017

,name,2017_dividend,2018,2019,2020
member_id,,,,,
1,Alice,7.61,3.80,1.90,1.91
2,Bob,103.12,51.56,25.78,25.78
3,Candice,55.62,27.81,13.90,13.91
4,Darwin,82.49,41.24,20.62,20.63
5,Eve,254.25,127.12,63.56,63.57
6,Federico,62.49,31.24,15.62,15.63
7,Gabi,12.50,6.25,3.12,3.13
8,HAL,41.51,20.76,10.38,10.37
9,Irene,251.78,125.89,62.94,62.95


In [84]:
allocations2017.sum()

name             AliceBobCandiceDarwinEveFedericoGabiHALIreneJules
2017_dividend                                               872.89
2018                                                        436.43
2019                                                         218.2
2020                                                        218.26
dtype: object

## Write a function to create a list of notices of allocation

In [94]:
allocations2015.columns

Index(['name', '2015_dividend', '2016', '2017', '2018'], dtype='object')

In [125]:
list(allocations2015.columns).index('2015_dividend')

1

In [116]:
allocations2015.columns[2:]

Index(['2016', '2017', '2018'], dtype='object')

In [117]:
[int(y) for y in allocations2015.columns[2:]]

[2016, 2017, 2018]

In [122]:
temp = allocations2015.drop(columns=['name', '2015_dividend'])
temp.columns

Index(['2016', '2017', '2018'], dtype='object')

In [113]:
allocations2015.loc[allocations2015['2015_dividend'] > 0,['name', '2016']]

,name,2016
member_id,,
1,Alice,6.22
2,Bob,24.57
3,Candice,34.81
4,Darwin,28.66
5,Eve,0.48


In [114]:
years = ['2016', '2017', '2018']
notices = [(allocations2015.loc[allocations2015['2015_dividend'] > 0, ['name', y]]
            .rename(columns={y: 'amount'})
            .assign(year_issued=2015, year_due=int(y), year_paid=np.NaN))
           for y in years]

notices_df = pd.concat(notices)
notices_df

,name,amount,year_issued,year_due,year_paid
member_id,,,,,
1,Alice,6.22,2015,2016,NaN
2,Bob,24.57,2015,2016,NaN
3,Candice,34.81,2015,2016,NaN
4,Darwin,28.66,2015,2016,NaN
5,Eve,0.48,2015,2016,NaN
1,Alice,3.11,2015,2017,NaN
2,Bob,12.28,2015,2017,NaN
3,Candice,17.40,2015,2017,NaN
4,Darwin,14.33,2015,2017,NaN


In [115]:
notices_df.reset_index()

,member_id,name,amount,year_issued,year_due,year_paid
0,1,Alice,6.22,2015,2016,NaN
1,2,Bob,24.57,2015,2016,NaN
2,3,Candice,34.81,2015,2016,NaN
3,4,Darwin,28.66,2015,2016,NaN
4,5,Eve,0.48,2015,2016,NaN
5,1,Alice,3.11,2015,2017,NaN
6,2,Bob,12.28,2015,2017,NaN
7,3,Candice,17.40,2015,2017,NaN
8,4,Darwin,14.33,2015,2017,NaN
9,5,Eve,0.24,2015,2017,NaN


In [ ]:
def get_notices_df(allocation_df):
    columns = allocation_df.columns